# COVID-19 Vaccine Adverse Reactions
Analysing adverse reactions caused by Covid Vaccines.
![Vaccine](https://citytoday.news/wp-content/uploads/2020/07/co-1-780x405.jpg)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
patient = pd.read_csv('../input/covid19-vaccine-adverse-reactions/2021VAERSDATA.csv', encoding='windows-1254')
vaccine = pd.read_csv('../input/covid19-vaccine-adverse-reactions/2021VAERSVAX.csv')
symptoms = pd.read_csv('../input/covid19-vaccine-adverse-reactions/2021VAERSSYMPTOMS.csv')

In [ ]:
patient.head()

In [ ]:
vaccine.head()

In [ ]:
new_patient_data = patient.merge(vaccine, on="VAERS_ID")
new_patient_data.head()

In [ ]:
new_patient_data['RECOVD'] = new_patient_data['RECOVD'].replace(np.nan, 'Unknown')
new_patient_data['RECOVD'] = new_patient_data['RECOVD'].replace('Y', 'Yes')
new_patient_data['RECOVD'] = new_patient_data['RECOVD'].replace('N', 'No')
new_patient_data['RECOVD'] = new_patient_data['RECOVD'].replace('U', 'Unknown')

# Analysis of Vaccines

In [ ]:
def plot_hist(df, col, xtitle, ytitle, title):
    fig = go.Figure(data=[go.Histogram(x=df[col])])
    fig.update_layout(title=title)
    fig.update_xaxes(title_text=xtitle)
    fig.update_yaxes(title_text=ytitle)
    fig.show()

### Manufacturers

In [ ]:
plot_hist(vaccine, 'VAX_MANU', 'Vaccine', 'Adverse Cases', 'Vaccine Manufacturers')

- **PFIZER\BIONTECH**'s vaccine has the most number of cases where recievers were reported to have adverse reaction.

### Recovery

In [ ]:
fig = px.histogram(new_patient_data, x="VAX_MANU", color="RECOVD")
fig.update_layout(
    title_text='Recovery After Adverse Reactions',
    xaxis_title_text='Vaccine', 
    yaxis_title_text='Adverse Cases'
)
fig.show()

- Even after the adverse reactions some recipients recovered.
- Its unclear from the data whether recipients are more likely to recover.

### Gender Analysis

- F : Female
- M : Male
- U : Unknown

In [ ]:
fig = px.histogram(new_patient_data, x="VAX_MANU", color="SEX")
fig.update_layout(
    title_text='Gender Distribution in Vaccines',
    xaxis_title_text='Vaccine', 
    yaxis_title_text='Adverse Cases'
)
fig.show()

- Large number of women reported of reactions from PFIZER\BIONTECH and MODERNA vaccines.

In [ ]:
fig = px.histogram(new_patient_data, x="SEX")
fig.update_layout(
    title_text='Gender Distribution Overall',
    xaxis_title_text='Gender', 
    yaxis_title_text='Adverse Cases'
)
fig.show()

- More than 50% of the recievers to experience adverse reaction from vaccines were females.
- Women are more likely to get adverse reaction from COVID vaccines.

### Vaccination Sites

In [ ]:
sites_count = new_patient_data['VAX_SITE'].value_counts()
sites = pd.DataFrame({'Site': sites_count.keys(), 'Count':sites_count})
fig = px.pie(sites, values="Count",names="Site", title="Reaction Report from Sites")
fig.update_layout(
    xaxis_title_text='Site',
    yaxis_title_text='Count'
)
fig.show()


- Most of adverse reaction cases were reported from LA.

### Vaccination Methods Used

- IM : Intramuscular Injection
- OT : Oral Route
- SYR: Syringe
- SC : Subcutaneous Injection
- UN : Unknown

In [ ]:
# Vaccination Route to Human Body
r = new_patient_data['VAX_ROUTE'].value_counts()
route = pd.DataFrame({'Route':r.keys(), 'Count': r.values})
route_fig = px.pie(route, values="Count", names="Route", title="Vaccination Route to Human Body")
route_fig.update_layout(
    title_text='Vaccination Route to Human Body',
    xaxis_title_text='Route', 
    yaxis_title_text='Count',
)
route_fig.show()

- 68% of the adverse reaction cases were given vaccines through Intramuscular Injection(IM).

# Age Distribution

In [ ]:
fig = px.scatter(new_patient_data, x="AGE_YRS", y="VAERS_ID", color="SEX")
fig.update_layout(
    title_text='Age Distribution and Gender',
    xaxis_title_text='Age',
    yaxis_title_text='ID'
)
fig.show()

In [ ]:
fig = px.scatter(new_patient_data, x="AGE_YRS", y="VAERS_ID", color="RECOVD")
fig.update_layout(
    title_text='Age Distribution and Recovery Status',
    xaxis_title_text='Age',
    yaxis_title_text='ID'
)
fig.show()

- Recipients between 20-70 years old are more likely to experience adverse reaction from covid vaccines.

# Symptom Analysis

In [ ]:
symptoms['SYMPTOM1'].isnull().sum()

In [ ]:
def create_wc(s="", h=15,w=45, title=""):
    wordcloud = WordCloud(height =1000,width = 3000).generate(s)
    plt.rcParams['figure.figsize'] = (h,w)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis("off")
    plt.show()

In [ ]:
symptom1 = " ".join(symptoms['SYMPTOM1'])

### Primary Symptoms

In [ ]:
create_wc(symptom1, title="Symptons of Vaccination")

In [ ]:
s2_dat = symptoms['SYMPTOM2'].dropna().value_counts()
symptom2 = pd.DataFrame({"Symptom":s2_dat[:10].keys(), "Count":s2_dat[:10].values})

In [ ]:
fig2 = px.bar(symptom2, x="Symptom", y="Count", title="Secondary Symptoms of Vaccination Reactions")
fig2.show()

In [ ]:
symptom2 = symptom2.append({"Symptom":"Other", "Count":s2_dat[10:].sum()}, ignore_index=True)

In [ ]:
fig3 = px.pie(symptom2, names="Symptom", values="Count", title="Secondary Symptoms of Vaccination Reactions")
fig3.show()

- Maximum number of recipients tested positive for SARS-CoV-2
- Vaccination resulted to death of 28 recipients.